In [1]:
import os

os.chdir('..')

In [2]:
from zipfile import ZipFile

import polars as pl

from aws.checksum import get_verified_aws_data_files
from aws.client_async import AwsClient
from aws.funding.parse import read_funding_csv
from config import BINANCE_DATA_DIR, TradeType

In [ ]:
trade_type = TradeType.um_futures
symbol = 'BTCUSDT'

funding_dir = BINANCE_DATA_DIR / 'aws_data' / AwsClient.TYPE_BASE_DIR[trade_type] / 'monthly' / 'fundingRate'
symbol_dir = funding_dir / symbol

verified_files = get_verified_aws_data_files(symbol_dir)
read_funding_csv(verified_files[0])

In [ ]:
parsed_dir = BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "funding"
funding_file = parsed_dir / f"{symbol}.pqt"

df = pl.read_parquet(funding_file)
df

In [ ]:
symbol = "BTCUSD_PERP"
trade_type = TradeType.cm_futures

api_dir = BINANCE_DATA_DIR / "api_data" / trade_type.value / "funding_rate"
df_funding = pl.read_parquet(api_dir / f"{symbol}.pqt")
df_funding